In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime

In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats,lngs)
lat_lngs

In [3]:
coordinates = list(lat_lngs)
print (coordinates)

[(-17.36883256570796, 24.87348489985598), (41.80170612796567, 164.9999116653937), (84.83632761009716, 171.43841927675226), (35.68599990458526, -64.52204407826083), (30.17870664833754, -116.21948422983652), (39.90003434098679, 55.52664461022556), (30.200226742858874, 110.5434791752408), (-55.89624973572833, 98.1345903462946), (82.91963856745292, 41.78263714312092), (74.9833885674272, 29.55755282451088), (21.763233633313703, 75.41058381416607), (-65.0903424364777, 85.74052962031169), (-22.43704398390446, -178.32166600117176), (-37.11273189115558, 76.66665891466613), (33.479244084391226, 42.60610922852186), (9.205282946568289, -97.18829999188398), (88.2172190161136, -165.2949851384928), (-81.42970573948618, 53.00577369717092), (-27.198265306464357, -25.81475336855786), (21.643644182029803, -155.76719740844962), (-48.89937517432881, -40.23925454930455), (-76.2449780021925, 169.79002444581465), (-83.52674483369276, -159.14299684231315), (1.6431624955637716, 170.17153777082802), (-27.1747934

In [4]:
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    if city not in cities:
        cities.append(city)
len(cities)

749

In [5]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
city_data = []
print("Beginning Data Retrieval        ")
print("--------------------------------")

record_count = 1
set_count = 1

for i, city in enumerate(cities):
        
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
 
    city_url = url + "&q=" + city.replace(" ","+")

    
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    
    record_count += 1
    
    
    try:
        
        city_weather = requests.get(city_url).json()
        
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']
        
        
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })


    except:
        print("City not found. Skipping...")
        pass


print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
  

Beginning Data Retrieval        
--------------------------------
Processing Record 1 of Set 1 | kasane
Processing Record 2 of Set 1 | severo-kurilsk
Processing Record 3 of Set 1 | pevek
Processing Record 4 of Set 1 | saint george
Processing Record 5 of Set 1 | san quintin
Processing Record 6 of Set 1 | gazanjyk
Processing Record 7 of Set 1 | yichang
Processing Record 8 of Set 1 | busselton
Processing Record 9 of Set 1 | ostrovnoy
Processing Record 10 of Set 1 | berlevag
Processing Record 11 of Set 1 | khargone
Processing Record 12 of Set 1 | vaini
Processing Record 13 of Set 1 | mahebourg
Processing Record 14 of Set 1 | hit
Processing Record 15 of Set 1 | pochutla
Processing Record 16 of Set 1 | barrow
Processing Record 17 of Set 1 | port alfred
Processing Record 18 of Set 1 | vila velha
Processing Record 19 of Set 1 | kahului
Processing Record 20 of Set 1 | chuy
Processing Record 21 of Set 1 | bluff
Processing Record 22 of Set 1 | mataura
Processing Record 23 of Set 1 | buariki
City 

In [7]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kasane,BW,-17.8167,25.1500,70.81,69,96,4.50,overcast clouds
1,Severo-Kurilsk,RU,50.6789,156.1250,25.29,85,51,5.06,broken clouds
2,Pevek,RU,69.7008,170.3133,-14.93,89,98,3.13,overcast clouds
3,Saint George,US,37.1041,-113.5841,50.63,15,0,21.85,clear sky
4,San Quintin,MX,30.4833,-115.9500,59.92,54,47,18.41,scattered clouds
5,Gazanjyk,TM,39.2446,55.5154,80.46,23,3,12.84,clear sky
6,Yichang,CN,30.7144,111.2847,60.58,94,100,1.63,overcast clouds
7,Busselton,AU,-33.6500,115.3333,57.02,88,0,9.91,clear sky
8,Ostrovnoy,RU,68.0531,39.5131,30.56,86,20,17.09,few clouds
9,Berlevag,NO,70.8578,29.0864,30.15,74,75,16.11,broken clouds


In [9]:
output_file = "weather_database/WeatherPy_Database.csv"
city_data_df.to_csv(output_file, index_label="City_ID")